In [1]:
import sys
sys.version

'2.7.12 (default, Nov 12 2018, 14:36:49) \n[GCC 5.4.0 20160609]'

# Data Process

In [2]:
#import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import joblib
#import scipy.misc
import tensorflow as tf
#from skimage.restoration import denoise_tv_chambolle
from tensorboardX import SummaryWriter

In [3]:
pp_path = '/home/obin/t2b_dataset/pickandplace/image'
data_list = np.sort([pp_path+'/'+x for x in os.listdir(os.path.join(pp_path,''))])
# pp_path = '/home/obin/t2b_dataset/pickandplace/temp/0202'
# data_list2 = np.sort([pp_path+'/'+x for x in os.listdir(os.path.join(pp_path,''))])
# data_list = np.hstack((data_list[0:-1],data_list2))
num_data = len(data_list)
print('num of data : ', num_data)

('num of data : ', 172)


In [4]:
def pretty_print_input(arr, ori = False):
    arr = np.squeeze(arr)
    if ori == True : 
        s_x, s_y, s_d, s_ori = arr[0:4]
        g_x, g_y, g_d, g_ori = arr[4:8]
        
        print('start pixel        goal pixek')
        print('x : %04f    x : %04f'%(s_x,g_x))
        print('y : %04f    y : %04f'%(s_y,g_y))
        print('z : %04f    z : %04f'%(s_d,g_d))
        print('o : %04f    o : %04f'%(s_ori,g_ori))
        print('\n')
    else : 
        s_x, s_y, s_d = arr[0:3]
        g_x, g_y, g_d = arr[3:6]
        
        print('start pixel        goal pixek')
        print('x : %04f    x : %04f'%(s_x,g_x))
        print('y : %04f    y : %04f'%(s_y,g_y))
        print('z : %04f    z : %04f'%(s_d,g_d))
        print('\n')
    
    
def pretty_print_out(arr):
    arr = np.squeeze(arr)
    s_px, s_py, s_pz, s_ox, s_oy, s_oz, s_ow = arr[0:7]
    g_px, g_py, g_pz, g_ox, g_oy, g_oz, g_ow = arr[7:14]
    
    print('start pose        goal pose')
    print('x : %04f    x : %04f'%(s_px,g_px))
    print('y : %04f    y : %04f'%(s_py,g_py))
    print('z : %04f    z : %04f'%(s_pz,g_pz))
    print('x : %04f    x : %04f'%(s_ox,g_ox))
    print('y : %04f    y : %04f'%(s_oy,g_oy))
    print('z : %04f    z : %04f'%(s_oz,g_oz))
    print('w : %04f    w : %04f'%(s_ow,g_ow))
    print('\n')
    


## depth image preprocess

In [5]:
#np.random.shuffle(data_list)
data_list

array(['/home/obin/t2b_dataset/pickandplace/image/0000.png',
       '/home/obin/t2b_dataset/pickandplace/image/0001.png',
       '/home/obin/t2b_dataset/pickandplace/image/0002.png',
       '/home/obin/t2b_dataset/pickandplace/image/0003.png',
       '/home/obin/t2b_dataset/pickandplace/image/0004.png',
       '/home/obin/t2b_dataset/pickandplace/image/0005.png',
       '/home/obin/t2b_dataset/pickandplace/image/0006.png',
       '/home/obin/t2b_dataset/pickandplace/image/0007.png',
       '/home/obin/t2b_dataset/pickandplace/image/0008.png',
       '/home/obin/t2b_dataset/pickandplace/image/0009.png',
       '/home/obin/t2b_dataset/pickandplace/image/0010.png',
       '/home/obin/t2b_dataset/pickandplace/image/0011.png',
       '/home/obin/t2b_dataset/pickandplace/image/0012.png',
       '/home/obin/t2b_dataset/pickandplace/image/0013.png',
       '/home/obin/t2b_dataset/pickandplace/image/0014.png',
       '/home/obin/t2b_dataset/pickandplace/image/0015.png',
       '/home/obin/t2b_d

In [6]:
## x_array = []
x_ori_array = []
y_array = []
mean_depth = 0
caution = []
# resume = int(input('where do you want to start from? : '))
resume = 0
for i in range(resume, num_data):
    img = cv2.imread(data_list[i])
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(imgray,70,255,0)

    image, contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    hull = []
    for c in contours :
        hull = cv2.convexHull(c)
        image = cv2.drawContours(img, hull, -1, 255, -1)
        cv2.imshow('image', image)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        
        (x, y) = np.array(c).sq
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        out = np.zeros_like(img)
        out = image[topx:bottomx+1, topy:bottomy+1]


        #cv2.drawContours(image,hull, 0,(0,255,0), 3)
        cv2.imshow('crop', out)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
    if key & 0xFF == ord('q') or key == 27:
        cv2.destroyAllWindows()
        break


    # Show the output image
    cv2.imshow('Output', out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


ValueError: too many values to unpack

In [9]:
c

array([[[254, 253]],

       [[253, 254]],

       [[247, 254]],

       [[248, 254]],

       [[249, 255]],

       [[255, 255]],

       [[255, 253]]], dtype=int32)

In [ ]:
curr_data = cv2.imread('/home/obin/t2b_dataset/pickandplace/image/0027.png')

In [ ]:
cv2.imshow('image', curr_data)
cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('/home/obin/t2b_dataset/pickandplace/image/0027.png')
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#threshold를 이용하여 binary image로 변환
ret, thresh = cv2.threshold(imgray,153,255,0)

#contours는 point의 list형태. 예제에서는 사각형이 하나의 contours line을 구성하기 때문에 len(contours) = 1. 값은 사각형의 꼭지점 좌표.
#hierachy는 contours line의 계층 구조
image, contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
image = cv2.drawContours(img, contours, -1, (0,255,0), 3)

cv2.imshow('image', image)
cv2.waitKey(50)
cv2.destroyAllWindows()

# Miscellaneous

In [ ]:
curr_data['pick'][0].x = 0.762535
curr_data['pick'][0].y = -0.087674
curr_data['pick'][0].z = -0.201498
curr_data['pick'][1].x = -0.02277
curr_data['pick'][1].y = 0.9879628
curr_data['pick'][1].z = -0.01236
curr_data['pick'][1].w = 0.1152506

In [ ]:
curr_data['pick']

In [ ]:
curr_data['place']

In [ ]:
joblib.dump(curr_data, os.path.join(pp_path,'/home/obin/t2b_dataset/pickandplace/data/0075.dat.gz'))